In [1]:

##### Import required packages for crawling data ######
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.request as ur
import urllib
import bs4
import csv
from googletrans import Translator
from datetime import datetime
import re
import os
import regex
import numpy as np

import time
from fuzzywuzzy import fuzz
from datetime import date
import json
import ast
try:
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen
import sys

from collections import OrderedDict
import tldextract
from collections import Counter
import operator

from selenium import webdriver
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
sys.setrecursionlimit(1000000000)
            

c:\users\v.behl\appdata\local\programs\python\python37\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [95]:
### UDF for various attributes been fetched from competitors website (to be updated with more)
def count_pictures(soup,flag):
    total_pictures = 0
    
    if flag == 'booking':
        if soup != '' and soup.find('div',{'class':'clearfix bh-photo-grid fix-score-hover-opacity'}):
            element = soup.find('div',{'class':'clearfix bh-photo-grid fix-score-hover-opacity'})

            upper_grid = len(element.find_all('div',{'aria-hidden':'true'}))

            lower_grid = len(element.find_all('div', {'class':'bh-photo-grid-thumb-cell'}))

            others =  re.findall(r'\d+',soup.find('span', {'class':'bh-photo-grid-thumb-more-inner-2'}).get_text())[0]

            total_pictures = int(upper_grid) + int(lower_grid) + int(others) -2


        if soup != '' and soup.find('div',{'class':'hp-gallery'}):

            element = soup.find('div',{'class':'hp-gallery'})

            total_pictures = len(element.find_all('img'))

    #if soup.find()

    return(total_pictures)

def review_ratings(soup,flag):
    review = ''
    rating = ''
    
    ###### Booking.com
    try:
        if flag =='booking':
            if soup.find('div',{'class': 'bui-review-score__badge'}):
                rating = soup.find('div',{'class': 'bui-review-score__badge'}).get_text().strip(" ")
            else:
                rating = "NA"
            if soup.find('div',{'class': 'bui-review-score__text'}):
                review = soup.find('div',{'class': 'bui-review-score__text'}).get_text().strip()
            else:
                review = "NA"
    except:
        review = "NA"
        rating = "NA"
        
     
    ####### vrbo.com
    try:
        if flag =='vrbo':
            if soup != '' and soup.find('div',{'class':'reviews-summary'}):
                review_raw_text =soup.find('div',{'class':'reviews-summary'}).find('strong',{'class':'reviews-summary__num-reviews text-link'}).get_text()
                review = re.findall(r'\d+',review_raw_text)[0]

                rating_raw_text = soup.find('div',{'class':'reviews-summary'}).find('span',{'class':'reviews-summary__badge'}).get_text()
                rating = re.findall(r'\d+',rating_raw_text)[0]

            else:
                rating ="NA"
                review = "NA"
    except:
        rating ="NA"
        review = "NA"
        

        
        
    return(rating,review)

def listing_price(soup,flag):
    price_clean = 0
    tax_amt = 0
    
    ### booking.com
    if flag == 'booking':
        if soup != '' and soup.find('div',{'class':'prco-wrapper bui-price-display prco-sr-default-assembly-wrapper prc-d-sr-wrapper'}):
            full_price =soup.find('div',{'class':'prco-wrapper bui-price-display prco-sr-default-assembly-wrapper prc-d-sr-wrapper'}).get_text() 
            full_price=   re.findall(r'\d+[-\.\s]??\d+',full_price.replace(",",""))
            #price_raw = soup.find('div',{'class':'bui-price-display__value prco-ltr-center-align-helper prco-font16-helper'}).get_text().strip(" ")
            price_clean =  full_price[0]
            #tax = soup.find('div',{'class':'prd-taxes-and-fees-under-price prco-inline-block-maker-helper on-hpage blockuid-453037801_193092061_2_0_0'}).get_text().strip(" ")
            tax_amt =  full_price[1]
    
    #### vrbo
    if flag == 'vrbo':
        if soup != '' and soup.find_all('meta'):
            #rent_amt = soup.find('span',{'class': 'rental-price__amount'}).get_text()
            price_clean = re.findall(r'[$]\d+',soup.find_all('meta')[3]['content'])[0].replace('$','')
            tax_amt = 0
    
    return(price_clean,tax_amt)


def member_since(soup,flag):
    member_since = 0
    if flag == "vrbo" and soup != '' :
        for ind_ in range(0,len(soup.find_all('script'))) :
            if soup.find_all('script')[ind_].contents[0].find('memberSince') >0:
                end = soup.find_all('script')[ind_].contents[0].find('memberSince') + len('memberSince')
                member_since = soup.find_all('script')[9].contents[0][end+3:end+7]
            
                break
    return(member_since)


def cancellation_policy(soup,flag):
    if flag =='booking':
        if soup != '' and soup.find('div',{'class':'hp-policies-block__cancellation-policies__policy hp-policies-block__cancellation-policies__policy--last'}):
            return(soup.find('div',{'class':'hp-policies-block__cancellation-policies__policy hp-policies-block__cancellation-policies__policy--last'}).get_text().strip(" ").replace("\n"," "))
        
        else:
            return("")
        
    if flag== 'vrbo':
        if soup != '' and soup.find('section',{'class':'house-rules__cancellation-policy'}).find('p'):
            return(soup.find('section',{'class':'house-rules__cancellation-policy'}).find('p').get_text().strip(" ").replace("\n"," "))
        else:
            return("")
    
    

In [101]:
### Main crawl function body 
## Step : 1. Hitting google search with listing name and city (These attributes will be directly available from the data)
## step : 2. Generating top 20 results and saving competitor URL that will refer us to desired page (currently modified for bookinng.com ana vrbo)
## step : 3. Hitting competitor URL and fectching info like review, rating, listing price, tax and number of images (can have more based on our requirments)
def get_competitor_data(name,city):
    
    vrbo_link = ''
    booking_link = ''
    total_pictures = ''
    review_bk = ''
    rating_bk = ''
    total_price = ''
    total_tax = ''
    total_price_bk = ''
    total_tax_bk = ''
    total_pictures_bk = ''
    
    #try:

    regex = re.compile(r" |,|-")
    name = regex.sub('+',name ).lower()
    city = regex.sub('+',city ).lower()

    results = 20

    user_agent='Chrome/35.0.1916.47 Chrome/57.0.2987.110 Chrome/61.0.3163.91 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/537.36'


    headers = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
                                        "Accept-Encoding": "gzip",
                                        "Accept-Language": "en-US,en;q=0.9,es;q=0.8",
                                        "Upgrade-Insecure-Requests": "1",
                                        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}


    url =  "https://www.google.com/search?q={}&num={}".format(name+"+"+city,results)

    print("google search url : ",url)


    req = requests.get(url,headers=headers,timeout=15)


#         delays = [7, 4, 6, 2, 10, 19]
#         delay = np.random.choice(delays)
    time.sleep(2)

    soup  =BeautifulSoup(req.content, 'lxml')

    vrbo_flag = 0 
    booking_flag = 0


    if soup.find_all('div', {'class':'g'}) :
        for link in soup.find_all('div', {'class':'g'}):
            web_link= link.find_all('a', href=True)[0]['href'].lower()
            #print(web_link,"\n")
            if 'maps/' not in web_link or 'search/' not in web_link:
                if 'vrbo' in web_link and vrbo_flag == 0:
                    vrbo_link = web_link
                    print("vrbo link : ",web_link)
                    vrbo_flag = 1
                if 'booking.com' in web_link and booking_flag == 0:
                    get_price = "?label=gen173nr-1DCAEoggI46AdIM1gEaGyIAQGYATG4ARfIAQzYAQPoAQH4AQKIAgGoAgO4As3Kne8FwAIB;sid=2d253dcdedcd11cb90e607e2d308697a;all_sr_blocks=453037801_193092061_2_0_0;bhgwe_cep=1;checkin=2020-01-20;checkout=2020-01-21;dest_id=20132415;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=1;highlighted_blocks=453037801_193092061_2_0_0;hp_group_set=0;hpos=1;no_rooms=1;room1=A%2CA;sb_price_type=total;selected_currency=USD;sr_order=popularity;srepoch=1575456368;srpvid=6adf4bb8dce30089;type=total;ucfs=1&#hotelTmpl"    

                    booking_link = web_link + get_price
                    booking_flag = 1
                    print("booking.com link : ",booking_link)

                if 'airbnb' in web_link :
                    print("airbnb link : ", web_link)

    if booking_link != '':
        booking_com_req = requests.get(booking_link,headers=headers,timeout=15)
        booking_com_soup= BeautifulSoup(booking_com_req.content, 'lxml')
    else:
        booking_com_soup = ''
    if vrbo_link != '' :
        vrbo_req = requests.get(vrbo_link,headers=headers,timeout=15)
        vrbo_soup = BeautifulSoup(vrbo_req.content, 'lxml')
    else:
        vrbo_soup = ''


    total_pictures_bk = count_pictures(booking_com_soup,'booking')
    review_bk,rating_bk = review_ratings(booking_com_soup,'booking')
    total_price_bk,total_tax_bk = listing_price(booking_com_soup,'booking')
    cp =  cancellation_policy(booking_com_soup,'booking')

    total_pictures_vrbo = count_pictures(vrbo_soup,'vrbo')
    review_vrbo,rating_vrbo = review_ratings(vrbo_soup,'vrbo')
    total_price_vrbo,total_tax_vrbo = listing_price(vrbo_soup,'vrbo')
    member_since_year = member_since(vrbo_soup,'vrbo')
    cp_vrbo =  cancellation_policy(vrbo_soup,'vrbo')
    #print(total_price,total_tax)

    df = pd.concat([pd.DataFrame([name],columns=['Name']),
                   pd.DataFrame([review_bk],columns=['review_booking_com']),
                   pd.DataFrame([rating_bk],columns=['rating_booking_com']),
                   pd.DataFrame([total_price_bk],columns=['total_price_per_night_booking_com_USD']),
                   pd.DataFrame([total_tax_bk],columns=['total_tax_per_booking_com_USD']),
                   pd.DataFrame([total_pictures_bk],columns=['total_pictures_booking_com']),
                   pd.DataFrame([cp],columns=['cancellation_policy_bk']),
                   pd.DataFrame([review_vrbo],columns=['review_vrbo']),
                   pd.DataFrame([rating_vrbo],columns=['rating_vrbo']),
                   pd.DataFrame([total_price_vrbo],columns=['total_price_vrbo']),
                   pd.DataFrame([total_tax_vrbo],columns=['total_tax_vrbo']),
                   pd.DataFrame([member_since_year],columns=['member_since_year']),
                   pd.DataFrame([cp_vrbo],columns=['cancellation_policy_vrbo'])], axis =1)

#     except:
#         df = pd.concat([pd.DataFrame([name],columns=['Name']),
#                        pd.DataFrame([review_bk],columns=['review_booking_com']),
#                        pd.DataFrame([rating_bk],columns=['rating_booking_com']),
#                        pd.DataFrame([total_price_bk],columns=['total_price_per_night_booking_com_USD']),
#                        pd.DataFrame([total_tax_bk],columns=['total_tax_per_booking_com_USD']),
#                        pd.DataFrame([total_pictures_bk],columns=['total_pictures_booking_com']),
#                        pd.DataFrame([cp],columns=['cancellation_policy_bk']),
#                        pd.DataFrame([review_vrbo],columns=['review_vrbo']),
#                        pd.DataFrame([rating_vrbo],columns=['rating_vrbo']),
#                        pd.DataFrame([total_price_vrbo],columns=['total_price_vrbo']),
#                        pd.DataFrame([total_tax_vrbo],columns=['total_tax_vrbo']),
#                        pd.DataFrame([member_since_year],columns=['member_since_year']),
#                        pd.DataFrame([cp_vrbo],columns=['cancellation_policy_vrbo'])], axis =1)
    
    return(df)

In [102]:
### Testing 
start_time = time.time()
df = get_competitor_data("11 Cabana Trail","")
print("--- %s seconds ---" % round((time.time() - start_time),2))
df 

google search url :  https://www.google.com/search?q=11+cabana+trail+&num=20
airbnb link :  https://www.airbnb.com/rooms/16392921
booking.com link :  https://www.booking.com/hotel/ro/pensiunea-felyviss-garda-de-sus.html?label=gen173nr-1DCAEoggI46AdIM1gEaGyIAQGYATG4ARfIAQzYAQPoAQH4AQKIAgGoAgO4As3Kne8FwAIB;sid=2d253dcdedcd11cb90e607e2d308697a;all_sr_blocks=453037801_193092061_2_0_0;bhgwe_cep=1;checkin=2020-01-20;checkout=2020-01-21;dest_id=20132415;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=1;highlighted_blocks=453037801_193092061_2_0_0;hp_group_set=0;hpos=1;no_rooms=1;room1=A%2CA;sb_price_type=total;selected_currency=USD;sr_order=popularity;srepoch=1575456368;srpvid=6adf4bb8dce30089;type=total;ucfs=1&#hotelTmpl
--- 6.33 seconds ---


Name review_booking_com rating_booking_com  \
0  11+cabana+trail                8.6          9 reviews   

   total_price_per_night_booking_com_USD  total_tax_per_booking_com_USD  \
0                                      0                              0   

   total_pictures_booking_com cancellation_policy_bk review_vrbo rating_vrbo  \
0                          34                                 NA          NA   

   total_price_vrbo  total_tax_vrbo  member_since_year  \
0                 0               0                  0   

  cancellation_policy_vrbo  
0

In [63]:
soup.find('section',{'class':'house-rules__cancellation-policy'}).find('p').get_text().strip(" ").replace("\n"," ")

'100% refund if canceled at least 30 days before arrival date. 50% refund if canceled at least 14 days before arrival date.'

In [29]:
### DO NOT RUN 


# row = []
# for a in range(1,6):
    

#     page_number = '/page/{}'.format(a)
#     print(page_number)
#     user_agent='Chrome/35.0.1916.47 Chrome/57.0.2987.110 Chrome/61.0.3163.91 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/537.36'


#     headers = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
#                                     "Accept-Encoding": "gzip",
#                                     "Accept-Language": "en-US,en;q=0.9,es;q=0.8",
#                                     "Upgrade-Insecure-Requests": "1",
#                                     "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}


#     url =  "https://community.withairbnb.com/t5/Hosting/complaint-against-airbnb/td-p/53691" + page_number

#     #print("google search url : ",url)


#     req = requests.get(url,headers=headers,timeout=15)


#     #         delays = [7, 4, 6, 2, 10, 19]
#     #         delay = np.random.choice(delays)
#     time.sleep(5)

#     soup  =BeautifulSoup(req.content, 'lxml')
    
#     counter = 0 
#     for para in soup.find_all('div',{'class':'lia-message-body-content'}):
    
#         if len(para.find_all('p')) ==1 :
#             row.append(para.find_all('p')[0].get_text())
#             counter = counter + 1
#         else:
#             for ind_ in range(0,len(para.find_all('p'))):
#                 row.append(para.find_all('p')[ind_].get_text())
#                 counter = counter + 1

    
#         print(counter)


/page/1
6
12
15
16
17
19
20
21
22
23
24
25
26
28
29
/page/2
6
7
8
9
10
11
12
13
14
15
16
17
18
20
21
22
/page/3
6
7
8
9
11
12
13
14
19
20
20
21
22
23
28
32
/page/4
6
8
17
18
19
20
21
22
23
25
28
29
30
32
33
38
/page/5
6
8
9
10
17
22
23
24
32
33
34
36
38
39
46
51


In [18]:
## DO NOT RUN

# page_number = '/page/{}'.format(1)
# print(page_number)
# user_agent='Chrome/35.0.1916.47 Chrome/57.0.2987.110 Chrome/61.0.3163.91 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/537.36'


# headers = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
#                                 "Accept-Encoding": "gzip",
#                                 "Accept-Language": "en-US,en;q=0.9,es;q=0.8",
#                                 "Upgrade-Insecure-Requests": "1",
#                                 "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}


# url =  "https://community.withairbnb.com/t5/Hosting/complaint-against-airbnb/td-p/53691" + page_number

# #print("google search url : ",url)


# req = requests.get(url,headers=headers,timeout=15)


# #         delays = [7, 4, 6, 2, 10, 19]
# #         delay = np.random.choice(delays)
# time.sleep(5)

# soup  =BeautifulSoup(req.content, 'lxml')

/page/1


### fetching host feedback from airbnb community page

In [53]:
def get_host_feedback(host_user_name):
    comments = []
    host_name = []
    
    
    def text_clean(x):
        col2 = re.sub(r'http\S+','', x,flags=re.MULTILINE)
        col2 = re.sub(r'@\S+','', col2,flags=re.MULTILINE)
        col2 = col2.lower()
        col2 = re.sub('[^a-zA-z0-9\s]',' ',col2)
        col2 = re.sub(r'\brt\b','',x)
        col2 = re.sub(' +',' ',x)
        col2 = col2.lstrip()
        #col2 = re.sub('\W+',' ', col2)
        col2 = col2.replace("\n","")
        col2 = col2.replace("\t","")
        col2 = col2.replace("\xa0","")
        return col2
    
    
    try:
        user_agent='Chrome/35.0.1916.47 Chrome/57.0.2987.110 Chrome/61.0.3163.91 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/537.36'


        headers = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
                                            "Accept-Encoding": "gzip",
                                            "Accept-Language": "en-US,en;q=0.9,es;q=0.8",
                                            "Upgrade-Insecure-Requests": "1",
                                            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}


        url =  "https://community.withairbnb.com/t5/forums/searchpage/tab/user?q={}&sort_by=login_sort".format(host_user_name)

        print("airbnb community page user specific : ",url)


        req = requests.get(url,headers=headers,timeout=15)


        #         delays = [7, 4, 6, 2, 10, 19]
        #         delay = np.random.choice(delays)

        soup  =BeautifulSoup(req.content, 'lxml')

        if soup.find('a',{'class':'lia-link-navigation user-post-count-link'})['href'] :
            post_url = "https://community.withairbnb.com" + soup.find('a',{'class':'lia-link-navigation user-post-count-link'})['href']
            print("Host post url : ",post_url)



        com_req = requests.get(post_url,headers=headers,timeout=15)

        comments_soup = BeautifulSoup(com_req.content, 'lxml')

        if len(comments_soup.find_all('div',{'class':'lia-message-body-content'})) >=1:

            total_comments = len(comments_soup.find_all('div',{'class':'lia-message-body-content'}))
            print(total_comments)


            for ind_ in range(0,total_comments):
                if comments_soup.find_all('div',{'class':'lia-message-body-content'})[ind_].find('div',{'class':'lia-full-body-container lia-js-hidden'}):
                    raw_text = comments_soup.find_all('div',{'class':'lia-message-body-content'})[ind_].find('div',{'class':'lia-full-body-container lia-js-hidden'}).get_text()
                elif comments_soup.find_all('div',{'class':'lia-message-body-content'})[ind_]:
                    raw_text = comments_soup.find_all('div',{'class':'lia-message-body-content'})[ind_].get_text()


                comments.append(text_clean(raw_text))
                host_name.append(host_user_name)

        else :
            comments.append("")
            host_name.append(host_user_name)
                                 


        df = pd.concat([pd.DataFrame(host_name,columns=['Host_name']),
                        pd.DataFrame(comments,columns=['comments'])], axis =1)
        
    except:
        print("check code")
        df = pd.concat([pd.DataFrame(host_name,columns=['Host_name']),
                pd.DataFrame(comments,columns=['comments'])], axis =1)



    
    
    return(df)

In [56]:
df = get_host_feedback('Theresa11')

airbnb community page user specific :  https://community.withairbnb.com/t5/forums/searchpage/tab/user?q=Theresa11&sort_by=login_sort
Host post url :  https://community.withairbnb.com/t5/forums/recentpostspage/post-type/message/user-id/41740
8


In [66]:
df

Host_name                                           comments
0  Theresa11  BYE BYE AIRBNB...NEVER AGAIN!!!  well.. guess ...
1  Theresa11  well.. guess what? I also just dialled their L...
2  Theresa11  I got 100$ superhost reward after remaining su...
3  Theresa11  super host $100 Reward disappeared and I was c...
4  Theresa11  Totally agree. WHen is Air BnB finally doing t...
5  Theresa11                                     Totally agree 
6  Theresa11  I agree on all of the points above. To add: Wh...
7  Theresa11  I have instant booking disabled and got a book...